In [35]:
from games.kuhn.kuhn3 import KuhnPoker3
from agents.counterfactualregret import CounterFactualRegret
from agents.agent_random import RandomAgent
from agents.mcts import MonteCarloTreeSearch
from agents.minimax import MiniMax
from collections import OrderedDict

# Kuhn Poker de 3 jugadores
El Kuhn póker es una versión simplificada del poker. Es un juego de apuestas con información incompleta, es decir, no conocemos las cartas de los contrincantes. Las reglas báscicas son:  

* **Mazo y reparto**  
    Se utiliza un mazo reducido, en el caso de tres jugadores se utilizan cuatro cartas distintas ('J', 'Q', 'K', 'A'). Se baraja y reparte una carta a cada jugador y se deja una abajo. Los jugadores solo conocen su carta.
* **Apuestas**  
    * **Apuesta inicial:** Antes de ver las cartas cada jugador hace una pequeña apuesta inicial al pozo.  
    * **Ronda de apuestas:** Luego de la apuesta inicial se juega una única ronda de apuestas en sentido horario.
        * El primer jugador puede pasar ("check") o apostar.  
        * Los próximos jugadores pueden irse ("fold") o igualar ("call").
* **Desenlace:** 
    Al terminar la ronda de apuestas si todos los jugadores se retiraron el jugador que es "mano" gana la partida. Si al final de la ronda hay más de un jugador se comparan las cartas y el jugador con la carta más alta gana. El ganador se lleva el bote.  

En definitiva, es un juego alternado, competitivo y de información incompleta ya que los agentes solo podrán observar su propia carta y las acciones de los demás. Esta información parcial es lo que se conoce como *Information set*.  
Su versión de dos jugadores es muy conocida y esta versión de tres jugadores también ha sido bastante explorada y en ***AParameterized Family of Equilibrium Profiles for Three-Player Kuhn Poker*** hallan analiticamente el equilibrio de Nash del juego.  

**Information sets terminales**  
Los information sets terminales son:   
**Profundidad 3**  
'ppp', Gana jugador 1, recompensa: 0 
'bpp', Gana jugador 1, recomepnsa: 0 
'bbp', Compiten jugadores 1 y 2, recompensa: 1   
'bpb', Compiten jugadores 1 y 3, recompensa: 2  
'bbb', Compiten jugadores 1, 2 y 3, recomepnsa: 3    
**Profundiad 4**  
'pbpp', Gana jugador 2   
'pbpb',   
'pbbp',   
'pbbb',  
**Profundiad 5**  
'ppbpp',   
'ppbbp',   
'ppbpb',   
'ppbbb'  

**Nota:** Los comentarios son asumiendo que el jugador que es mano (comienza jugando) es el jugador 1. Si la mano cambia, también los jugadores anotados.   

In [36]:
g = KuhnPoker3()

In [40]:
"""
# MiniMax seed=None, depth: int=sys.maxsize
MonteCarloTreeSearch simulations: int=100, rollouts: int=10, selection: Callable[[MCTSNode], MCTSNode]=uct, max_depth=None, eval_name: str="eval"
# RandomAgent seed=None
# CounterFactualRegret 
"""

classes_parameters = {
    'agent_0': [MiniMax,      {'seed': 42, 'depth': 3}],
    'agent_1': [CounterFactualRegret,      {}],
    'agent_2': [RandomAgent, {}]
}

my_agents = {}
g.reset()
for agent_id in g.agents:
    AgentClass, params = classes_parameters[agent_id]
    my_agents[agent_id] = AgentClass(game=g, agent=agent_id, **params)


In [41]:
counter_facutal_agents_id = []
for agent in g.agents:
    if isinstance(my_agents[agent], CounterFactualRegret):
        print('Training agent ' + agent)
        counter_facutal_agents_id.append(agent)
        my_agents[agent].train(3000)
        print('Agent ' + agent + ' policies:')
        print(OrderedDict(map(lambda n: (n, my_agents[agent].node_dict[n].policy()), sorted(my_agents[agent].node_dict.keys()))))
        print('')
    print('Agent ' + agent + " doesn't need training.")

Agent agent_0 doesn't need training.
Training agent agent_1
Agent agent_1 policies:
OrderedDict([('0', array([0.99865772, 0.00134228])), ('0b', array([0.00938338, 0.99061662])), ('0bb', array([0.5, 0.5])), ('0bp', array([0.99867021, 0.00132979])), ('0p', array([0.99798928, 0.00201072])), ('0pb', array([0.00398936, 0.99601064])), ('0pbb', array([0.5, 0.5])), ('0pbp', array([0.99899261, 0.00100739])), ('0pp', array([0.99867021, 0.00132979])), ('0ppb', array([0.4406638, 0.5593362])), ('0ppbb', array([0.5, 0.5])), ('0ppbp', array([0.99865772, 0.00134228])), ('1', array([0.94392833, 0.05607167])), ('1b', array([0.99744246, 0.00255754])), ('1bb', array([0.5, 0.5])), ('1bp', array([0.00620347, 0.99379653])), ('1p', array([0.92497006, 0.07502994])), ('1pb', array([0.97181186, 0.02818814])), ('1pbb', array([0.5, 0.5])), ('1pbp', array([0.77911373, 0.22088627])), ('1pp', array([0.95587052, 0.04412948])), ('1ppb', array([0.9947657, 0.0052343])), ('1ppbb', array([0.5, 0.5])), ('1ppbp', array([0.40

In [42]:
cum_rewards = dict(map(lambda agent: (agent, 0.), g.agents))
niter = 2000
for _ in range(niter):
    g.reset()
    turn = 0
    while not g.done():
        #print('Turn: ', turn)
        #print('\tPlayer: ', g.agent_selection)
        #print('\tObservation: ', g.observe(g.agent_selection))
        a = my_agents[g.agent_selection].action()
        #print('\tAction: ', g._moves[a])
        g.step(action=a)
        turn += 1
    #print('Rewards: ', g.rewards)
    for agent in g.agents:
        cum_rewards[agent] += g.rewards[agent]
print('Average rewards:', dict(map(lambda agent: (agent, cum_rewards[agent]/niter), g.agents)))


Average rewards: {'agent_0': -0.3455, 'agent_1': 0.76, 'agent_2': -0.4145}


In [43]:
print('Check learned policies against theoretical policies:')

Check learned policies against theoretical policies:


In [45]:
if len(counter_facutal_agents_id) > 0:
    JX_b = my_agents[counter_facutal_agents_id[0]].node_dict['0'].policy()[1]
    print(f'Agent: 0 - Hand: J_ - History: [] - Probability of betting: {JX_b}')

    QX_pb_b = my_agents[counter_facutal_agents_id[0]].node_dict['1pb'].policy()[1]
    print(f'Agent: 0 - Hand: Q_ - History: pb - Probability of betting: {QX_pb_b} - Theoretic value: {JX_b+1/3} -  Difference: {abs(QX_pb_b - (JX_b+1/3))}')

    KX_b = my_agents[counter_facutal_agents_id[0]].node_dict['2'].policy()[1]
    print(f'Agent: 0 - Hand: K_ - History: [] - Probability of betting: {KX_b} - Theoretic value: {3 * JX_b} -  Difference: {abs(KX_b - 3 * JX_b)}')

    XJ_p_b = my_agents[counter_facutal_agents_id[0]].node_dict['0p'].policy()[1]
    print(f'Agent: 0 - Hand: _J - History: p - Probability of betting: {XJ_p_b} - Theoretic value: {1/3} -  Difference: {abs(XJ_p_b - 1/3)}')

    XJ_p_b = my_agents[counter_facutal_agents_id[0]].node_dict['0p'].policy()[1]
    print(f'Agent: 0 - Hand: _J - History: p - Probability of betting: {XJ_p_b} - Theoretic value: {1/3} -  Difference: {abs(XJ_p_b - 1/3)}')

    XQ_b_b = my_agents[counter_facutal_agents_id[0]].node_dict['1b'].policy()[1]
    print(f'Agent: 0 - Hand: _Q - History: b - Probability of betting: {XQ_b_b} - Theoretic value: {1/3} -  Difference: {abs(XQ_b_b - 1/3)}')


Agent: 0 - Hand: J_ - History: [] - Probability of betting: 0.0013422818791946308
Agent: 0 - Hand: Q_ - History: pb - Probability of betting: 0.028188136658131715 - Theoretic value: 0.3346756152125279 -  Difference: 0.30648747855439623
Agent: 0 - Hand: K_ - History: [] - Probability of betting: 0.011222587000782717 - Theoretic value: 0.004026845637583892 -  Difference: 0.007195741363198825
Agent: 0 - Hand: _J - History: p - Probability of betting: 0.0020107238605898124 - Theoretic value: 0.3333333333333333 -  Difference: 0.3313226094727435
Agent: 0 - Hand: _J - History: p - Probability of betting: 0.0020107238605898124 - Theoretic value: 0.3333333333333333 -  Difference: 0.3313226094727435
Agent: 0 - Hand: _Q - History: b - Probability of betting: 0.0025575447570332483 - Theoretic value: 0.3333333333333333 -  Difference: 0.3307757885763001
